### 지니차트 Top 200

In [86]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

- 1. 데이터 가져오기

In [87]:
url = 'https://www.genie.co.kr/chart/top200'
result = requests.get(url)
result.text

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [88]:
# Chrome User-Agent
header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
result = requests.get(url, headers=header)
soup = BeautifulSoup(result.text, 'html.parser')

In [89]:
trs = soup.select('tr.list')
len(trs)

50

In [90]:
# 개발자 도구에서 찾는 법
# #body-content > div.newest-list > div > table > tbody > tr:nth-child(1) > td.info
trs = soup.select('#body-content > div.newest-list > div > table > tbody > tr')
len(trs)

50

- 3. 하나를 선택해서 원하는 정보 추출

In [91]:
tr = trs[0]

In [92]:
# rank
rank = tr.select_one('.number').get_text().split('\n')[0].strip()
rank

'1'

In [93]:
# imp url
img = 'https:' + tr.select_one('.cover > img')['src']
img

'https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/083/665/559/83665559_1681116738103_1_140x140.JPG/dims/resize/Q_80,0'

In [94]:
title = tr.select_one('.title.ellipsis').get_text().strip()
title

'I AM'

In [95]:
# 19금 노래 제목
tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()

'I AM'

In [96]:
# 위의 두 코드를 합치기
# 방법 1)
tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
# 방법 2)
try:
    title = title = tr.select_one('.title.ellipsis').get_text().strip()
except:
    tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()

In [97]:
artist = tr.select_one('.artist.ellipsis').string.strip()
artist

'IVE (아이브)'

In [98]:
album = tr.select_one('.albumtitle.ellipsis').string.strip()
album

"I've IVE"

##### 4. 한 페이지 내 정보 가져오기

In [99]:
# 방법 1) 딕셔너리의 리스트로 가져오기
line = []
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0].strip()
    img = 'https:' + tr.select_one('.cover > img')['src']
    title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
    artist = tr.select_one('.artist.ellipsis').string.strip()
    album = tr.select_one('.albumtitle.ellipsis').string.strip()
    line.append({'rank':rank, 'img':img, 'title':title, 'artist':artist, 'album':album})
df1 = pd.DataFrame(line)
df1.head()

,rank,img,title,artist,album
0,1,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,I AM,IVE (아이브),I've IVE
1,2,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,Kitsch,IVE (아이브),I've IVE
2,3,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,꽃,지수 (JISOO),ME
3,4,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,Ditto,NewJeans,NewJeans 'OMG'
4,5,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,CHRISTIAN,Zior Park,WHERE DOES SASQUATCH LIVE? PART 1


In [100]:
rank_list, img_list, title_list, artist_list, album_list = [], [], [], [], []
for tr in trs:
    rank_list.append(tr.select_one('.number').get_text().split('\n')[0].strip())
    img_list.append('https:' + tr.select_one('.cover > img')['src'])
    title_list.append(tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip())
    artist_list.append(tr.select_one('.artist.ellipsis').string.strip())
    album_list.append(tr.select_one('.albumtitle.ellipsis').string.strip())
df2 = pd.DataFrame(
    'rank': rank_list, 'img': img_list, 'title': title_list, 'artist': artist_list, 'album': album_list
)
df2.head()

SyntaxError: invalid syntax (1056916586.py, line 9)

- 5. 에러 처리

- 모든 페이지에 대해 처리

In [ ]:
#https://www.genie.co.kr/chart/top200?ditc=D&ymd=20230424&hh=13&rtm=Y&pg=1
from datetime import datetime
now = datetime.now()
now

datetime.datetime(2023, 4, 24, 15, 16, 9, 512962)

In [ ]:
ymd = now.strftime('%Y%M%d') # ymd = f'{now.year}{now.month:02d}{now.day:02d}'
hh = now.strftime('%H')      # hh = f'{now.how:02d}'
ymd, hh

('20231624', '15')

In [ ]:
from tqdm import tqdm

lines = []
base_url = 'https://www.genie.co.kr/chart/top200?ditc=D'
for page in tqdm(range(1,5)):
    url = f'{base_url}&{ymd}&{hh}&pg={page}'
    result = requests.get(url, headers=header)
    soup = BeautifulSoup(result.text, 'html.parser')
    trs = soup.select('tr.list')
    for tr in trs:
        rank = tr.select_one('.number').get_text().split('\n')[0].strip()
        img = 'https:' + tr.select_one('.cover > img')['src']
        title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
        artist = tr.select_one('.artist.ellipsis').string.strip()
        album = tr.select_one('.albumtitle.ellipsis').string.strip()
        lines.append({'rank':rank, 'img':img, 'title':title, 'artist':artist, 'album':album})


100%|██████████| 4/4 [00:03<00:00,  1.29it/s]


- 7. 데이터프레임 만들기

In [ ]:
df = pd.DataFrame(lines)
df.tail()

,rank,img,title,artist,album
195,196,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,Dive,NCT 도재정,Perfume - The 1st Mini Album
196,197,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,너를 생각해,주시크 (Joosiq),너를 생각해
197,198,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,언제나 사랑해,케이시 (Kassy),조영수 리메이크 프로젝트 Part.2
198,199,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,시작,가호 (Gaho),이태원 클라쓰 OST Part.2 (JTBC 금토드라마)
199,200,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,서랍,10CM,그 해 우리는 OST Part.1


In [ ]:
df.to_csv(f'data/{ymd}{hh}_지니차트.csv', index=False)